In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("truehealth/medqa")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.jsonl:   0%|          | 0.00/10.4M [00:00<?, ?B/s]

dev.jsonl:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1272 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 10178
    })
    validation: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 1272
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx'],
        num_rows: 1273
    })
})


In [ ]:
import json

train_data = dataset['train']

with open('medqa_train.jsonl', 'w') as f_out:
    for example in train_data:
        prompt = example['question']
        output = example['answer']
        json_line = json.dumps({"instruction": prompt, "output": output})
        f_out.write(json_line + '\n')


In [ ]:
with open('medqa_val.jsonl', 'w') as f_out:
    for example in dataset['validation']:
        prompt = example['question']
        output = example['answer']
        json_line = json.dumps({"instruction": prompt, "output": output})
        f_out.write(json_line + '\n')


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Configuration
MODEL_NAME = "gpt2"
TRAIN_FILE = "medqa_train.jsonl"
VAL_FILE = "medqa_val.jsonl"
OUTPUT_DIR = "./medqa_gpt2_finetuned"
BATCH_SIZE = 4
EPOCHS = 3
MAX_LENGTH = 1024

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

# GPT-2 has no pad token by default — set it to eos token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

# Load dataset
data_files = {"train": TRAIN_FILE, "validation": VAL_FILE}
dataset = load_dataset("json", data_files=data_files)

# Tokenization function
def tokenize_fn(example):
    prompt = example["instruction"] + "\nAnswer: " + example["output"]
    tokens = tokenizer(prompt, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_datasets = dataset.map(tokenize_fn, batched=False)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    prediction_loss_only=True,
    fp16=torch.cuda.is_available(),
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Train the model
trainer.train()

# Save the final model and tokenizer
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

Map:   0%|          | 0/10178 [00:00<?, ? examples/s]

Map:   0%|          | 0/1272 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.330700,0.331681
2,0.326900,0.317503
3,0.307900,0.313499


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


('./medqa_gpt2_finetuned/tokenizer_config.json',
 './medqa_gpt2_finetuned/special_tokens_map.json',
 './medqa_gpt2_finetuned/vocab.json',
 './medqa_gpt2_finetuned/merges.txt',
 './medqa_gpt2_finetuned/added_tokens.json',
 './medqa_gpt2_finetuned/tokenizer.json')

In [ ]:
from huggingface_hub import login

login("PUT_YOUR_HUGGINGFACE_TOKEN")

In [ ]:
from huggingface_hub import HfApi, create_repo
from transformers import AutoTokenizer, AutoModelForCausalLM

repo_name = "gpt2-medqa-ft"  # change if needed

# Create a repo on the hub (skip if it already exists)
create_repo(repo_name, exist_ok=True)

# Save model & tokenizer locally
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Push to hub
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aranya31/gpt2-medqa-ft/commit/98d8a51f587c6c256a9875e18f92e70452920919', commit_message='Upload tokenizer', commit_description='', oid='98d8a51f587c6c256a9875e18f92e70452920919', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Aranya31/gpt2-medqa-ft', endpoint='https://huggingface.co', repo_type='model', repo_id='Aranya31/gpt2-medqa-ft'), pr_revision=None, pr_num=None)